In [1]:
%pip install pandas
%pip install tornado_helper --upgrade --no-cache-dir --find-links=. --force-reinstall --no-deps

Note: you may need to restart the kernel to use updated packages.
Looking in links: .
  Attempting uninstall: tornado_helper
    Found existing installation: tornado_helper 1.0.3
    Uninstalling tornado_helper-1.0.3:
      Successfully uninstalled tornado_helper-1.0.3
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd 

df = pd.read_csv("catalog.csv")

In [16]:
# Convert 'start_time' to datetime
df['start_time'] = pd.to_datetime(df['start_time'])
df = df[df['start_time'].dt.year == 2017]

df.describe()

,start_time,lat,lon,event_id,Year,Julian Day,Hour
count,20072,20072.000000,20072.000000,2.007200e+04,20072.0,20072.000000,20072.000000
mean,2017-06-09 18:57:36.102530816,36.067602,-91.068722,9.051336e+05,2017.0,160.247708,12.519629
min,2017-01-01 22:35:00,23.703960,-126.071502,6.645010e+05,2017.0,1.000000,0.000000
25%,2017-04-16 20:10:00,31.158947,-96.473026,7.065970e+05,2017.0,106.000000,4.000000
50%,2017-06-15 22:16:00,35.725748,-91.502895,1.078024e+06,2017.0,166.000000,14.000000
75%,2017-08-01 22:51:00,40.261210,-83.730841,1.079113e+06,2017.0,213.000000,20.000000
max,2017-12-20 18:01:00,49.955617,-66.257176,1.079911e+06,2017.0,354.000000,23.000000
std,NaN,5.236421,9.389057,1.879218e+05,0.0,70.019233,8.112673


In [7]:
df = df[["start_time", "lat", "lon", "event_id"]]

In [10]:
df["Year"] = df["start_time"].dt.year
df["Julian Day"] = df["start_time"].dt.strftime("%j").astype(int)  # Convert to Julian day
df["Hour"] = df["start_time"].dt.hour
df.head()

,start_time,lat,lon,event_id,Year,Julian Day,Hour
59457,2017-01-01 22:35:00,31.647191,-84.811088,1077881,2017,1,22
59458,2017-01-01 22:42:00,32.306650,-85.149992,1077881,2017,1,22
59459,2017-01-01 22:56:00,31.106899,-84.160352,1077881,2017,1,22
59460,2017-01-01 23:09:00,32.218102,-87.071869,1077881,2017,1,23
59461,2017-01-01 23:23:00,31.770340,-85.692839,1077881,2017,1,23


In [11]:
unique_time_tuples = set(df[["Year", "Julian Day", "Hour"]].itertuples(index=False, name=None))
len(unique_time_tuples)

1888

In [ ]:

import boto3
from botocore import UNSIGNED
from botocore.config import Config

s3_client = boto3.client("s3", config=Config(signature_version=UNSIGNED))

bucket_name = "noaa-goes16"
prefix_template = "ABI-L2-MCMIPC/{year}/{julian_day:03d}/{hour:02d}/"

matching_files = []

count = 0
for year, julian_day, hour in unique_time_tuples:
    count += 1
    print(f"Evaluating {count} - {year}, {julian_day}, {hour}")
    
    prefix = prefix_template.format(year=year, julian_day=julian_day, hour=hour)
    
    # List objects in S3 bucket matching the prefix
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if "Contents" in response:
        for obj in response["Contents"]:
            print("Match!")
            matching_files.append(obj["Key"])

Evaluating 1 - 2017, 149, 21
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Evaluating 2 - 2017, 96, 15
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Evaluating 3 - 2017, 86, 1
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Evaluating 4 - 2017, 127, 2
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Evaluating 5 - 2017, 233, 20
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Evaluating 6 - 2017, 225, 16
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Evaluating 7 - 2017, 150, 22
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Evaluating 8 - 2017, 191, 23
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Match!
Evaluating 9 - 2017, 142, 18
Match!
Match!
Match!
Match!
Match!
Match!
Evaluating 10 - 2017, 183, 19
Match!

In [15]:
len(matching_files)

20015